In [1]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("EnglishTenseUniqueDataset.xlsx")

# Display first few rows
df.head()

,Sentence,Labels
0,by 2050 ai architects will have designed selfc...,Future
1,in the future sustainable transportation optio...,Future
2,china has been actively involved in peacekeepi...,Present
3,educational diversity is a hallmark of foreign...,Present
4,the coach substituted an underperforming player,Past
